In [ ]:
from pathlib import Path
import metadata, navigation
import numpy as np
import json
import datetime as dt
import pytz 

In [ ]:
from pysolar import solar

### When to run this notebook

After stacking VNIR and SWIR cubes into full supercubes `[flightline]_VNIR_SWIR_supercube.geo.bsq`, and assembling full-supercubes `[flightline]_VNIR_SWIR_supercube.geo_sca.bsq` (for example using notebook 06). There are a number of ways to accomplish this, among them:

* Manually stacking VNIR and SWIR output using ENVI
* Stacking VNIR and SWIR output programmatically with the `gdal` tools
* Using PARGE's integrated processing to produce a single supercube

The code in the following is quite flexible and needs to be run thoughtfully. With manual stacking in ENVI as originally described, we would be using bands 1-170 VNIR and 2-288 SWIR for a total of 457 bands. With integrated processing, cutting over at 954 nm from VNIR to SWIR, we end up with 459 bands.

The goal of this code is to produce suitable metadata for use in ATCOR and prepare final packaging metadata. We also save the navigation and solar parameters in a text file to be loaded when needed.  

In [ ]:
projdir = Path(r"Z:\fihyper\cwaigl\20210803_BC")
prefix = '20210803-BC'

### Load reference data 

**This section is for prototyping. For production use, go straight to the next section.** Let's first test with one line

In [ ]:
lineno = '07' 

Depending on the settings in NEO HySpex RAD, the file names for spectral radiance may or may not have a name element '\_rad'. 

In [ ]:
extra = '_rad'
# extra = ''
referencefile = f'{prefix}_{lineno}_VNIR_1800_SN00812_FOVx2_raw{extra}_bsq_float32.hdr'

Enter the subdirectory that holds the PARGE output files. We will use them as reference. 

In [ ]:
exampledir = projdir / f"{prefix}_{lineno}/RAD"

Load metadata from refrerence file

In [ ]:
refmeta = metadata.hdrfile_to_dict(exampledir / referencefile)
refmeta

{'description': '{',
 'Frameperiod': '12000',
 'Integration time': '8300',
 'Binning': '1',
 'Number of frames': '26866',
 'Aperture size': '0.008000',
 'dw': '1',
 'EQ': '0',
 'FOVexp': '5',
 'Lens': '5',
 'NumberOfAvg': '1',
 'CalibAvailable': '0',
 'Number of background': '200',
 'Pixelsize x': '0.000320',
 'Pixelsize y': '0.000640',
 'ID': 'VNIR_1800_SN00812',
 'Comment': '',
 'Serialnumber': '812',
 'Scanningmode': 'Airborne }',
 'samples': '1800',
 'lines': '26866',
 'bands': '182',
 'header offset': '5246976',
 'acquisition date': '2021-07-02',
 'acquisition start time': '13:22:21',
 'data type': '4',
 'data ignore value': '2',
 'interleave': 'bsq',
 'default bands': '{75,46,19}',
 'byte order': '0',
 'wavelength': '{410.359,413.529,416.698,419.868,423.038,426.207,429.377,432.547,435.716,438.886,442.055,445.225,448.395,451.564,454.734,457.904,461.073,464.243,467.412,470.582,473.752,476.921,480.091,483.261,486.43,489.6,492.769,495.939,499.109,502.278,505.448,508.618,511.787,514.9

Get timestamp from reference metadata and convert to UTC.  Assumption: Data is aquired during AKDT. Check timestamp whether this makes sense!

In [ ]:
#origts = dt.datetime.strptime(refmeta['acquisition time'], '%Y-%m-%dT%H:%M:%S.0Z')
origts = dt.datetime.strptime(f"{refmeta['acquisition date']} {refmeta['acquisition start time']}", '%Y-%m-%d %H:%M:%S')
origts = origts + dt.timedelta(hours=8)
print(origts)

2021-07-02 21:22:21


Load navigation data and extract average heading, average flight height, andthe time stamp from the NAV data.

In [ ]:
navdir = projdir / f"{prefix}_{lineno}/NAV"
navpath = navdir / f"{prefix}_{lineno}_VNIR_1800_SN00812_FOVx2_raw.txt"

In [ ]:
data1 = np.loadtxt(navpath, dtype={'names': ('line', 'lon', 'lat', 'height', 'roll', 'pitch', 'heading', 'tstamp'), 
                                   'formats': ('<i4', '<f8', '<f8', '<f8', '<f8', '<f8', '<f8', '<f8')})
middle = len(data1)//2

In [ ]:
flight_heading = navigation.avg_angle(data1['heading'])
flight_elevation = data1['height'].mean()
flight_lat = data1['lat'][middle]
flight_lon = data1['lon'][middle]

print("Flightline heading: {:3.2f} °\nFlightline elevation: {:4.2f} m".format(flight_heading, flight_elevation))
print("Flightline latitude: {:3.2f} °\nFlightline longitude: {:3.2f} °".format(flight_lat, flight_lon))

Flightline heading: 170.85 °
Flightline elevation: 2406.44 m
Flightline latitude: 64.90 °
Flightline longitude: -150.79 °


In [ ]:
navts = dt.timedelta(seconds=data1['tstamp'][middle])
hours = navts.seconds // 60 // 60
minutes = (navts.seconds - hours*3600)//60

print(navts)

corr = 0
if hours == 0 and origts.hour == 23:
    corr = 1
elif origts.hour != hours:
    print("!!! CAREFUL: Something odd is going on with the timestamps. Check your data for consistency.",
         "Do not continue just running the code !!! ")

5 days, 21:33:43.765230


In [ ]:
datestamp = dt.datetime(origts.year, origts.month, origts.day, hours, minutes, navts.seconds % 60) + dt.timedelta(days=corr)
datestamp = pytz.utc.localize(datestamp)

print(datestamp)

2021-07-02 21:33:43+00:00


In [ ]:
azimuth = solar.get_azimuth(flight_lat, flight_lon, datestamp)

C:\Users\cwaigl\AppData\Local\Continuum\miniconda3\envs\hyspex_proc\lib\site-packages\pysolar\solartime.py:112: UserWarning: I don't know about leap seconds after 2018
  (leap_seconds_base_year + len(leap_seconds_adjustments) - 1)


In [ ]:
zenith = 90 - solar.get_altitude_fast(flight_lat, flight_lon, datestamp)

In [ ]:
print("Solar azimuth: {:3.2f} °\nSolar zenith: {:3.2f} °".format(azimuth, zenith))

Solar azimuth: 168.47 °
Solar zenith: 42.22 °


### Add navigation data file to each line directory

In [ ]:
projdir = Path(r"Z:\fihyper\cwaigl\20210803_BC")
prefix = '20210803-BC'

Setting parameters - edit if needed

In [ ]:
subdirstr = 'NAV'
navfilepattern = "_VNIR_1800_SN00812_FOVx2_raw.txt"
refdirstr = 'RAD'
extra = '_rad'
#extra = ''
reffilepattern = f'_VNIR_1800_SN00812_FOVx2_raw{extra}_bsq_float32.hdr'
flightlines = list(projdir.glob(f"{prefix}_??"))
print(f"There are {len(flightlines)} flightlines to process")

There are 7 flightlines to process


We're writing files to the `NAV` subdirectory. By default, writing is disabled. Edit filename pattern as desired. 

In [ ]:
write_files = True
outfile_patt = "VNIR_SWIR_rad_geo_flightdata.txt"
outdirstr = subdirstr

In [ ]:
for linedir in flightlines:
    linenumstr = linedir.parts[-1][-2:]
    print(f"Working on line number {linenumstr}")
    # load reference data and retrieve timestamp
    refpath = linedir / refdirstr / f"{prefix}_{linenumstr}{reffilepattern}" 
    refmeta = metadata.hdrfile_to_dict(refpath)
    origts = dt.datetime.strptime(
        f"{refmeta['acquisition date']} {refmeta['acquisition start time']}", 
        '%Y-%m-%d %H:%M:%S') + dt.timedelta(hours=8)
    # load navigation data
    navpath = linedir / subdirstr / f"{prefix}_{linenumstr}{navfilepattern}"
    navdata = np.loadtxt(navpath, dtype={'names': ('line', 'lon', 'lat', 'height', 'roll', 'pitch', 'heading', 'tstamp'), 
                                   'formats': ('<i4', '<f8', '<f8', '<f8', '<f8', '<f8', '<f8', '<f8')})
    middle = len(navdata)//2
    flight_heading = navigation.avg_angle(navdata['heading'])
    flight_roll = navdata['roll'].mean()
    flight_pitch = navdata['pitch'].mean()
    flight_roll_std = navdata['roll'].std()
    flight_pitch_std = navdata['pitch'].std()
    flight_elevation = navdata['height'].mean()
    flight_lat = navdata['lat'][middle]
    flight_lon = navdata['lon'][middle]
    navts = dt.timedelta(seconds=navdata['tstamp'][middle])
    hours = navts.seconds // 60 // 60
    minutes = (navts.seconds - hours*3600)//60
    corr = 0
    if hours == 0 and origts.hour == 23:
        corr = 1
    elif hours-origts.hour not in [0, 1]:
        print("!!! CAREFUL: Something odd is going on with the timestamps. Check your data for consistency.",
             "Do not continue just running the code !!! ")
        print(hours, origts.hour)
        continue
    # make a new corrected timestamp
    datestamp = dt.datetime(
        origts.year, origts.month, origts.day, 
        hours, minutes, navts.seconds % 60) + dt.timedelta(days=corr)
    datestamp_akdt = datestamp + dt.timedelta(hours=-8)
    datestamp = pytz.utc.localize(datestamp)
    # calcualte solar geometry with new timestamp
    azimuth = solar.get_azimuth(flight_lat, flight_lon, datestamp)
    zenith = 90 - solar.get_altitude_fast(flight_lat, flight_lon, datestamp)
    #populate flightdata dictionary
    if write_files: 
        flightdata = {}
        flightdata['flightlinename'] = linedir.parts[-1]
        flightdata['origts_utc'] = origts.strftime('%Y-%m-%dT%H:%M:%S.0Z')
        flightdata['linets_utc'] = datestamp.strftime('%Y-%m-%dT%H:%M:%S.0Z')
        flightdata['linets_akdt'] = datestamp_akdt.strftime('%Y-%m-%d %H:%M:%S')
        flightdata['heading_avg'] = round(flight_heading, 2)
        flightdata['roll_avg'] = round(flight_roll, 2)
        flightdata['roll_std'] = round(flight_roll_std, 2)
        flightdata['pitch_avg'] = round(flight_pitch, 2)
        flightdata['pitch_std'] = round(flight_pitch_std, 2)
        flightdata['elevation_m_amsl'] = round(flight_elevation, 2)
        flightdata['latitude'] = round(flight_lat, 2)
        flightdata['longitude'] = round(flight_lon, 2)
        flightdata['sun_azimuth'] = round(azimuth, 2)
        flightdata['sun_zenith'] = round(zenith, 2)
        outfp = linedir / outdirstr / f"{prefix}_{linenumstr}_{outfile_patt}"
        with open(outfp, "w") as dst:
            dst.write(json.dumps(flightdata, indent=2))
        print(f"Done writing {outfp}.")
    # output results
    print("\tFlightline heading: {:3.2f} °\n\tFlightline elevation: {:4.2f} m".format(flight_heading, flight_elevation))
    print("\tFlightline roll: {:3.2f} °\n\tFlightline roll std: {:3.2f} °".format(flight_roll, flight_roll_std))
    print("\tFlightline pitch: {:3.2f} °\n\tFlightline pitch std: {:3.2f} °".format(flight_pitch, flight_pitch_std))
    print("\tFlightline latitude: {:3.2f} °\n\tFlightline longitude: {:3.2f} °".format(flight_lat, flight_lon))
    print(f"\tNEO start timestamp (UTC): {origts}\n\tFlightline timestamp (UTC): {datestamp}")
    print(f"\tFlightline timestamp (AKDT): {datestamp_akdt}")
    print("\tSolar azimuth: {:3.2f} °\n\tSolar zenith: {:3.2f} °".format(azimuth, zenith))
    print()

Working on line number 01


C:\Users\cwaigl\AppData\Local\Continuum\miniconda3\envs\hyspex_proc\lib\site-packages\pysolar\solartime.py:112: UserWarning: I don't know about leap seconds after 2018
  (leap_seconds_base_year + len(leap_seconds_adjustments) - 1)


Done writing Z:\fihyper\cwaigl\20210803_BC\20210803-BC_01\NAV\20210803-BC_01_VNIR_SWIR_rad_geo_flightdata.txt.
	Flightline heading: 4.58 °
	Flightline elevation: 1796.58 m
	Flightline roll: 0.14 °
	Flightline roll std: 2.68 °
	Flightline pitch: -1.57 °
	Flightline pitch std: 1.31 °
	Flightline latitude: 64.72 °
	Flightline longitude: -148.29 °
	NEO start timestamp (UTC): 2021-08-03 20:58:25
	Flightline timestamp (UTC): 2021-08-03 21:09:35+00:00
	Flightline timestamp (AKDT): 2021-08-03 13:09:35
	Solar azimuth: 164.01 °
	Solar zenith: 48.09 °

Working on line number 02
Done writing Z:\fihyper\cwaigl\20210803_BC\20210803-BC_02\NAV\20210803-BC_02_VNIR_SWIR_rad_geo_flightdata.txt.
	Flightline heading: 175.97 °
	Flightline elevation: 1762.89 m
	Flightline roll: 0.45 °
	Flightline roll std: 2.53 °
	Flightline pitch: -0.34 °
	Flightline pitch std: 1.12 °
	Flightline latitude: 64.72 °
	Flightline longitude: -148.31 °
	NEO start timestamp (UTC): 2021-08-03 21:03:04
	Flightline timestamp (UTC): 2